파이썬 컬렉션 처리 함술ㄹ 함수형 프로그래밍의 관점에서 살펴본다.

반복 가능 객체나 시퀀스를 재귀나 명시적 루프를 사용해 처리ㅏ기 위한 추가 디자인 패턴을 살펴본다. 그리고 제너레이터 식에서 데이터의 컬렉션에 대해 scalar() 함수를 적용하는 방법을 살펴본다.

### 함수의 다양성에 대한 정리

함수를 두 가지 넓은 부류로 구별해야 한다.
* 스칼라 함수는 개별 값에 적용할 수 있고, 개별적인 결과를 내놓는다. abs(), pow() 등의 함수나 math 모듈에 있는 모든 함수들이 스칼라 함수의 예다.

* Collection 함수는 반복 가능한 컬렉션에 작용한다. 

컬렉션 함수는 다음과 같이 세 가지로 나눌 수 있다.

* reduction: 컬렉션에 있는 여러 값을 함수를 사용해 겹쳐 호출함으로써 최종적으로 단일 값을 만들어 낸다. aggregate 함수라고도 부른다. 
* mapping: 함수를 컬렉션의 모든 원소에 적용한다. 결과는 입력 컬렉션과 크기가 같은 다른 컬렉션이다.
* filter: 컬렉션의 모든 원소에 함수를 적용하여 빌부 원소는 버리고 일부 원소를 통과시키는 것이다. 

### 반복 가능 객체로 작업하기

보통 for 루프를 사용해 컬렉션에 대한 작업을 수행한다. 튜플, 리스트, 맵, 집합 등의 실체화한 컬렉션에 대해 작업하는 경우, for 루프는 상태를 명시적으로 관리한다. 이는 순수한 함수형 프로그래밍으로부터 벗어난 것이지만, 파이썬에 필요한 최적화를 반영하는 것이기도 하다. for문을 평가하는 과정에서 생기는 반복자 객체에만 상태가 한정되도록 보장할 수 있다면, 순수한 함수형 프로그래밍에서 크게 벗어나지 않은 상태에서 for문을 활용할 수 있을 것이다. 예를 들어 for 루프의 변수를 그 루프에 대해 들여쓴 본문의 바깥에서 사용할 경우 순수한 함수형 프로그래밍에서 상당히 멀리 벗어난 것이다.

for 루프를 사용하는 일반적인 응용으로는 풀기(처리(감싸기(반복 가능객체))) 디자인 패턴을 들 수 있다. 감싸기() 함수는 반복 가능 객체의 각 원소를 정렬하기 위한 키나 다른 값과 함께 2-튜플로 묶는다. 그 후 이 2-튜플을 바탕으로 처리를 수행한다. 마지막으로, 풀기() 함수는 감싸기 위해 사용했던 키 값을 버리고, 원래의 원소를 복원한다.

이러한 작업은 함수형 문맥에서 매우 자주 잇어나는 일이기 때문에 이러한 경우에 자주 사용하는 두 함수가 존재한다.

In [1]:
fst = lambda x: x[0]
snd = lambda x: x[1]

두 가지 모두 처리() 나 풀기() 함수에서 편리하게 사용할 수 있다.

다른 일반적인 패턴은 감싸기(감싸기(감싸기()))다. 이경우, 간단한 튜플에서 시작하여 다른 결과를 추가한 후 좀 더 크고 복잡한 튜플을 만든다. 이러한 패턴을 변경한 것 중 흔히 사용하는 것으로는 확장(확장(확장()))이 있다. 여기서는 원래의 튜플을 감싸는 대신 한 번 확장할 때마다 새로운 namedtuple 인스턴스를 만들어낸다. 이 두 가지 모두 첨가 디자인 패턴이라 부를 수 있다.

첨가 디자인 패턴을 위도와 경도 값을 다루기 위해 사용할 수 있다. 첫 단계는 어떤 경로에 속한 (lat, lon) 이라는 단순한 지점 구간을 나타내는 (begin, end) 쌍으로 만드는 것이다. 각 쌍은((lat, lon) (lat, lon))가 될 것이다.

다음 절에서 파일의 내용에 대한 제너레이터 함수를 만드는 방법을 볼 것이다. 이 반복 가능 객체에는 우리가 처리할 입력 데이터가 들어갈 것이다.

데이터를 준비하고 나면, 그 이후의 절에서는 각 구간에 대한 하버사인 거리를 구하도록 구간을 감싸는 방법을 살펴본다. 이렇게 감싸기(감싸기(감싸기())) 패턴으로 처리한 마지막 결과는 3-튜플((lat-lon), (lat-lon), distance)의 시퀀스다. 그 후 이를 분석하여 최장거리나 최단거리, 어떤 경로를 둘러싼 직사각형 또는 데이터를 요약한 다른 값 등을 구할 수 있다.

### XML 파일 구문 분석하기

XML[확장 가능한 마크업 언어] 파일을 구문 분석하여 위경도 쌍을 가져오는 것부터 시작할 것이다. 이를 통해 파이썬에서 확실히 팜수형이 아닌 기능을 감싸 값의 반복 가능한 시퀀스를 만드는 과정을 보여줄 것이다. xml.etree 모듈을 사용할 것이고, 구문 분석한 결과인 ElementTree 객체에는 모든 값에 대해 방문할 수 있는 findall() 메서드가 들어 있다.

다음과 같은 XML 엘리먼트를 찾을 것이다.
~~~
<Placemark><Point>
     <coodinates>-76.33029518659048,37.54901619777347,0</coordinates>
</point></Placemark>
~~~

파일에는 ` <Placemark> ` 태그가 여럿 들어 있다. 각각에는 Point가 있고, 그 내부에는 좌표 구조가 들어 있다. 이는 지리 정보가 담겨 있는 전형적인 키홀 마크업 언어의 예다.

XML 파일을 구문 분석하는 것은 두 가지 추상화 수준을 거친다. 하위 수준에서는 여러 태그, 애트리뷰트 값, 그리고 XML 파일 내부의 내용을 팢는다. 상위 수준에서는 텍스트나 애트리뷰트 값으로부터 유용한 객체를 만들어 낸다.

하위 수준 처리는 다음과 같이 수행할 수 있다.

In [2]:
import xml.etree.ElementTree as XML
def row_iter_kml(file_obj):
    na_map = {
        "ns0" : "http://www/.opengis.net/kml/2.2",
        "ns1" : "http://www/.opengis.net/kml/ext/2.2"
    }
    doc = XML.parse(file_obj)
    return (comma_split(coordinates.text)
            for coordinates in 
            doc.findall("./ns0:Document/ns0:Folder/ns0:Placemark/ns0:Point/ns0:coordinates", ns_map))

이 함수는 보통 with 문 안에서 이미 열려 있는 파일을 받는다. 하지만 XML 구문 분석기가 처리할 수 있는 파일과 유사한 객체라면 어떤 것이든 관계 없다. 이 함수에는 간단하고 정적인 dict 객체, ms_map이 들어 있고, 그 객체에는 우리가 찾고자 하는 XML 태그의 namespace 매핑 정보가 들어 있다. 이 딕셔너리를 XML의 ElemenTree.findall() 메서드에서 사용할 것이다.

구문 분석의 핵심은 doc.findall()으로 찾은 태그의 시퀀스를 사용하는 제네레이터 함수에 있다. 이 태그의 시퀀스를 comme_split() 함수로 처리하여 텍스트 값을 콤마로 구분한 구성 요소로 나눈다.

comma_split() 함수는 문자열의 split() 메서드를 함수형으로 만든 것으로, 이는 다음과 같다.

In [3]:
def comma_split(text):
    return text.split(",")

문법적인 균일성을 강조하기 위해 메서드를 함수로 둘러쌌다.

이 함수가 만들어 내는 결과는 데이터행의 반복 가능한 시퀀스다. 각 행에는 경로에 속하는 각 지점을 이루는 세 가지 문자열, 즉 위도, 경도, 고도가 들어간다. 이 3-튜플은 아직 유용하지 않다. 우리는 부동 소수점 수의 값으로 바꾸는 처리를 좀 더 진행해야 한다.

이러한 식으로 저수준의 파싱 결과를 값의 반복 가능한 시퀀스로 내놓는 방식을 사용하면 여러 종류의 테이터 파일을 단순하고 일관성 있게 처리할 수 있다.

### 파일을 상위 수준에서 구문 분석하기

저수준 구문 분석을 마쳤다면, 원데이터를 재구성하여 파이썬 프로그램에서 유용한 그 무언가로 만들어 낼 수 있다. 데이터를 직렬화할 수 있는 XML, JSON, CSV 외의 여러 다양한 물리적 형식에 대해 이러한 재구성을 적용할 수 있다.

우리의 목표는 작은 제네레이터 함수를 사용해 분석한 데이터를 애플리케이션이 사용하기 알맞은 형태로 변환하는 것이다. 제네레이터 함수는 row_iter_kml() 함수가 찾아낼 수 있는 텍스트에 적용 가능한 몇 가지 간단한 변환을 포함한다. 이러한 변환은 다음과 같다.

* altitude를 없애고, 필요하면 latitude나 longitude만을 남기는 것
* 순서를 (longitude, latitude)에서 (latitude, longitude)로 바꾸는 것

다음과 같이 도구 함수를 정의하면 더 일관된 구문을 사용해 이 두가지 변환을 다룰 수 있다.

In [4]:
def pick_lat_lon(lon, lat, alt):
    return lat, lon

def lat_lon_kml(row_iter):
    return (pick_lat_lon(*row) for row in row_iter)

이 함수는 pick_lat_lon() 함수를 각 줄에 적용할 것이다. 우리는 *row를 사용해 각 줄의 튜플에 있는 세 원소를 따로따로 pick_lat_lon() 함수의 인자로 넘겼다. 그 함수는 이제 3-튜플에서 우리에게 필요한 두 값을 취하여 순서를 바꿀 것이다.

좋은 함수형 설계를 사용하면 어떤 함수를 그와 동등한 다른 함수로 언제든지 바꿀 수 있다는 사실을 알아두는 것이 중요하다. 따라서 리팩토링이 매우 쉽다. 우리는 여러 함수에 대한 다양한 구현을 제공할 때 이렇나 목표를 당성하려고 노력해 왔다. 똑똑한 함수형 언어 컴파일러라면 최적화 과정에서 함수를 이러한 식으로 대치할 수 있다.

다음과 같은 처리를 사용해 파일을 분석하고 원하는 구조를 만들 것이다.

In [5]:
# import urllib

# with urllib.request.urlopen("file:./Winter%202012-2013.kml") as source:
#     v1 = tuple(lat_lon_kml(row_iter(source)))
# print(v1)

### 시퀀스의 원소를 둘씩 짝짓기

데이터 재구성에 있어 일반적인 요구사항 중 하나는 시퀀스에 있는 여러 점의 정보부터 시작점-끝점 쌍을 만드는 것이다. 주어진 시퀀스 $S={s_0, s_1, s_2, ... ,s_n}$에 대해 쌍의 시퀀스 $S ={(s_0, s_1), (s_1, s_2), ... , (s_{n-1},s_n)}$을 만들고 싶다. 시계열 분석을 수행하는 경우에는 좀 더 많은 개수의 정보를 묶어야 할 수도 있다. 여기에서는 단지 연속된 두 값만을 묶을 것이다.

쌍의 시퀀스에 대해 haversine 함수를 적용하면 각 쌍의 시작점에서 끝점에 이르는 거리를 계산할 수 있다. 그래픽 애플리케이션에서는 이러한 기법을 사용해 여러 점으로 이뤄진 경로를 일련의 선분 세그먼트들로 바꿀 수 있다.

왜 원소를 둘씩 묶어야 할까? 다음과 같이 하면 안될까?

In [6]:
def iters():
    for i in range(10):
        yield i

def compute_something(a, b):
    return a, b

iterable = iters()

begin = next(iterable)
for end in iterable:
    compute_something(begin, end)
    begin = end

이는 데이터의 각 부분을 시작-끝 쌍으로 처리한다. 하지만 처리 함수와 데이터를 재구성하는 루프가 너무 밀접하게 엮여 있다. 따라서 재사용이 필요 이상으로 복잡해진다. 쌍을 만들어 내는 알고리즘과 compute_something()이 엮여 있기 때문에 쌍을 만들어 내는 알고리즘을 따로 떼어 테스트하기도 어렵다.

이러한 식으로 조합된 함수를 사용하면, 애플리케이션을 재설정할 수 있는 가능성도 줄어든다. 또 위 코드는 명시적인 상태 begin을 사용하기 때문에 일이 더 복잡해질 가능성도 있다. loop의 몸통에 기능을 추가하는 경우, 어떤 점을 미처 고려하지 못하여 begin 변수를 제대로 설장하는 것을 잊어버릴 수 있다. 또한 filter() 함수로 인해 begin 변수를 제대로 변경할 수 업게 만드는 오류를 발생시킬 수 있는 if문이 들어갈 수도 있다.

단순한 쌍 만들기 함수를 분리하면 재사용성을 더 높일 수 있다. 목표는 장기적으로 재사용성을 높이는 것이다. 쌍 만들기 함수와 같은 기본적인 기능을 다수 제공하는 라이브러리를 만들어 낸다면, 문제를 더욱 빠르고 자신 있게 처리할 수 있다.

재귀를 사용하면 순수 함수적인 방식으로 쌍을 만들 수 있다. 다음은 경로상의 점을 둘씩 묶는 함수 버전 중 하나다.

In [7]:
def pairs(iterable):
    def pair_from(head, iterable_tail):
        nxt = next(iterable_tail)
        yield head, nxt
        yield from pair_from(nxt, iterable_tail)
    try:
        return pair_from( next(iterable_tail), iterable)
    except StopIteration:
        return

필수적인 함수는 내부의 pair_from() 함수다. 이 함수는 반복 가능 객체의 머리에 있는 우너소와 반복 가능 객체 자체에 작용한다. 그 함수는 첫 번째 쌍을 내놓은 후 반복 가능 객체에서 다음 번 원소를 빼내고 자기 자신을 재귀적으로 호출하여 다른 쌍을 계속 만들어 내게 한다.

이 함수를 pairs() 함수에서 호출했다. pairs() 함수는 초기화를 제대로하고, 종료를 표현하는 예외를 조용히 처리한다.

꼬리 호출 재귀를 최적화하기 위한 전략은 재귀를 제너레이터 식으로 바꾸는 것이다. 이 방식을 사용하면 단순한 for 루프로 최적화할 ㅅ ㅜ있다. 다음은 경로에 있는 점의 쌍을 돌려주는 함수의 다른 구현이다.

In [8]:
def legs(lat_lon_iter):
    begin = next(lat_lon_iter)
    for end in lat_lon_iter:
        yield begin, end
        begin = end

이 하수는 꽤 빠르고 스택의 한계에도 영향을 받지 않느다. 이 코드는 시퀀스 제너레이터가 발생시키는 모든 것으로부떠 쌍을 만들기 때문에 시퀀스의 타입과 무관하게 잘 작동한다. 루프 내부에 다른 처리 함수가 없기 때문에 이 legs() 함수를 필요에 따라 재사용할 수 있다.

이 함수를 다음과 같은 쌍의 시퀀스를 발생시키는 것처럼 생각할 수 있다.

list[0:1], list[1:2], list[2:3], .... , list[-2:]

다음은 이 함수를 다른 방식으로 정리한 것이다.

zip(list, list[1:])

이해하는 데 도움이 되지만, 이 두가지 코드는 오직 시퀀스 객체에 대해서만 작동한다. legs()와 pairs() 함수는 시퀀스 객체를 포함해 모든 반복 가능 객체에 대해 잘 작동한다.

### iter() 함수를 명시적으로 사용하기

순수하게 함수적인 관점에서는 만든 모든 반복 가능 객체들을 재귀함수로 처리할 수 있어야 하며, 상태는 재귀호출 스택뿐이어야 한다. 실용적으로 살펴보면, 파이썬의 반복 가능 객체들은 오직 다른 for 루프의 평가에만 참여한다. 일반적인 경우의 예로는 컬렉션과 반복 가능 객체가 있다. 컬렉션을 다루는 경우 for문은 반복자 객체를 만들어 낸다. 제너레이터 함수를 다루는 경우에는 제너레이터 함수가 반복자이며, 그 내부에 상태를 저장한다. 파이썬의 프로그래밍 관점에서 볼 때 이 둘은 서로 동등한 경우가 많다. 하지만 next() 함수를 꼭 호출해야만 하는 경우에는 그 둘이 완전히 동등하지 않다.

legs() 함수는 반복 가능 객체에서 첫 원소를 얻기 위해 next()를 명시적으로 호출했다. 제너레이터 함수, 제너레이터 식 또는 다른 반복 가능 객체의 경우, 이는 잘 작동한다. 하지만 튜플이나 리스트와 같은 시퀀스 객체에서는 잘 작동하지 않는다.

다음은 next()와 iter() 함수의 사용법은 명확히 보여주는 세 가지 예다.


In [9]:
list(legs(x for x in range(3)))

[(0, 1), (1, 2)]

In [10]:
list(legs([0, 1, 2]))

TypeError: 'list' object is not an iterator

In [11]:
list(legs(iter([0, 1, 2])))

[(0, 1), (1, 2)]

첫 번째는 legs() 함수를 반복 가능 객체에 적용했다. 이 경우 반복 가능 객체는 제너레이터 식이었다. 이는 이번 장 앞에서 살펴본 예제와 마찬가지로 우리가 예상한 것과 똑같은 결과를 내놓는다. 원소가 둘씩 잘 짝지어졌고, 점이 3개인 경로에서 2개의 부분 경로가 만들어졌다. 

두 번째는 legs() 함수를 시퀀스에 적용했다. 결과는 오루다. for문에서 사용하는 경우에는 list 객체와 반복 가능 객체가 동등하지만 어디서나 그러한 것은 아니다. 시퀀스는 반복자가 아니기 때문에 next() 함수를 제공하지 안흔ㄴ다. 하지만 for문은 시퀀스에 반복자를 자동으로 만들기 때문에 리스트를 매끄럽게 잘 처리한다. 

두 번째가 제대로 동작하기 위해 명시적은 list 객체에서 반복자를 만들어야 한다. 그렇게 하면 legs() 함수가 리스트의 첫 원소를 반복자를 통해 가져올 수 있다.

### 단순한 루프 확장하기

단순한 루프에 넣을 수 있는 확장에는 두 가지가 있다. 먼저 filter 확장을 살펴본다. 그 경우에는 값을 제외하여 더 이상 고려 대상이 되지 못하게 만들 수 있을 것이다. 제외할 값은 데이터의 이상치나 형식이 잘못된 원본 데이터일 수 있다. 그 후 원본에서 새로운 객체를 만들어 내는 간단한 변환을 수행해 원본 데이터를 매핑할 수 있다. 이는 string을 float로 변환하는 것이다. 하지만 단순한 루프를 매핑으로 확장한다는 아이디어 자체를 다양한 상황에 적용할 수 있다. 우리는 조금 전에 살펴본 pair() 함수를 리팩토링할 것이다. 어떤 값을 제외시키기 위해 점의 순서를 조장해야 한다면 어떻게 해야 할까? 이 경우 일부 데이터 값을 제외시킬 수 있는 filter 확장이 필요해질 것이다.

루프에서는 복잡도는 최소화하기 위해 애플리케이션에 따른 추가 처리를 수행하지 않고 쌍(튜플)만을 반환할 것이다. 단순성을 유지하면 처리 하태를 혼동할 사능성도 더욱 적어진다.

루프 설계에 filter 확장을 추가하면 다음과 같다.

In [13]:
def legs_filter(lat_lon_iter):
    begin = next(lat_lon_iter)
    for end in lat_lon_iter:
        if end == 0:# some rule for rejecting
            continue
        yield begin, end
        begin = end

이에는 특정 값을 거부하는 처리를 집어넣었다. 루프가 간결하고 이해하기 쉽기 때문에 처리가 제대로 될 것이라는 확신을 할 수 있다. 또한 결과가 모든 반복 가능 객체에 대해 작동하기 때문에 만들어진 쌍을 최종적으로 어디에 사용할 것인지와 관계 없이 이 함수에 대한 테스트를 쉽게 작성할 수 있다.

다음 리팩토링은 루프에 새로운 매핑을 추가할 것이다. 설계가 발전해 나감에 따라 매핑이 추가되는 일이 흔하다. 우리의 경우에는 string의 시퀀스를 가지고 있다. 이를 나중에 사용하기 위해 float 값으로 바꿀 필요도 있다. 이는 상대적으로 단순한 매핑이지만 디자인 패턴을 보여줄 수 있다.

다음은 이러한 데이터 매핑을 처리하는 한 가지 방법이다. 제너레이터 함수를 돌러싼 제너레이터 식을 사용한다.

In [18]:
lat_lon_al = [(1, 2, 3), (1, 2, 3), (1, 2, 3), (1, 2, 3)]
print(tuple(legs((float(lat), float(lon)) for lat, lon in lat_lon_kml(lat_lon_al))))

(((2.0, 1.0), (2.0, 1.0)), ((2.0, 1.0), (2.0, 1.0)), ((2.0, 1.0), (2.0, 1.0)))


legs() 함수를 제너레이터 식에 적용한 후 lat_lon_kml()의 결과를 가지고 float 값을 만들었다. 이 코드는 뒤에서 앞으로 읽을 수 있다. lat_lon_kml()의 출력을 float 값의 쌍으로 변환 후 이를 legs() 함수의 시퀀스로 변환한다.

이러한 식의 코드는 금방 복잡해진다. 여기서도 내포된 함수가 매우 많다. float(), legs(), tuple() 함수를 제너레이터에 적용하고 있다. 복잡한 식을 리팩토링하는 일반적인 방법 중 하나는 제너레이터 식과 실체화한 컬렉션을 분리하는 것이다.

다음과 같이 단순화할 수 있다.

In [23]:
fit = ((float(lat), float(lon)) for lat, lon in lat_lon_kml(lat_lon_al))

In [24]:
print(tuple(legs(fit)))

(((2.0, 1.0), (2.0, 1.0)), ((2.0, 1.0), (2.0, 1.0)), ((2.0, 1.0), (2.0, 1.0)))


fit라는 이름의 변수에 제너레이터 함수를 대입했다. 객체를 만들기 위해 list 내장을 사용하지 않았기 떄문에 이 변수는 컬레겻내 객체가 아니다.